This is a demo code for building a RAG based chatbot with Gradio as UI

**Core Functionality:**

* RAG chatbot: Retrieve relevant documents, augment prompt, generate response.
* LLM for response : gpt-4o-mini (Need to use OpenAI API and ensure this model name is correct).
* Vector Database: FAISS (in-memory).
* Document Upload & Processing: Upload documents, chunk them, embed them, store in FAISS.
* Short-term memory: Chat history within the session.
* Text and Voice Input: Gradio supports both.

**UI Requirements (Gradio):**
* Neat and Clean UI.
* Left Pane for Document Upload with a link (file upload).
* "Process Document" Button below upload.
Chatbot interface in the main area.
* "Submit" button for generating response.
* Output display on screen.

**Technical Requirements:**
* Langchain for RAG and LLM interaction.
* OpenAI API for LLM access.
* FAISS for vector storage.
* Gradio for UI.
* Modular code.


### Install Required Libraries:

In [ ]:
!pip install langchain openai faiss-cpu gradio pypdf langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing in

Set up OpenAI API Key:

In [ ]:
from google.colab import userdata
import openai

openai.api_key  = userdata.get('OPENAI_API_KEY')

#export OPENAI_API_KEY="YOUR_OPENAI_API_KEY"

## Code Structure (Modular Approach)

In [ ]:
# Import all required libraries and functions.
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader  # For PDF document loading
from langchain.memory import ConversationBufferMemory

### Document Processing Functions

In [ ]:
def load_document(file_path):
    """Load document from file path."""
    loader = PyPDFLoader(file_path) # Assuming PDF for now, can extend for other types
    document = loader.load()
    return document

def chunk_document(document):
    """Chunk document into smaller parts."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(document)
    return chunks

def create_vector_store(chunks, embeddings):
    """Create FAISS vector store from document chunks."""
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

### Chatbot Functions

In [ ]:
def initialize_chatbot(vector_store):

    """Initialize the RAG chatbot with vector store and memory."""
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, api_key=openai.api_key) # Or gpt-3.5-turbo if gpt-4o-mini is not directly available

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    retriever = vector_store.as_retriever()

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )
    return qa_chain

def get_chatbot_response(qa_chain, query):
    """Get response from the chatbot."""
    response = qa_chain.invoke({"question": query})
    return response['answer']

def clear_chat_history():
    """Function to clear chat history (to be implemented in UI interaction)."""
    return None # Placeholder, actual clear logic will depend on Gradio state management.


### Main Functionality

In [ ]:
#Main function to set up embeddings, UI, and event handlers.

embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key) # Initialize OpenAI embeddings

vector_store_instance = None # To hold the vector store instance
qa_chain_instance = None     # To hold the chatbot QA chain instance

def process_document(file_list):
  global vector_store_instance, qa_chain_instance             # Allows modification of these vars
  print("Process Document button clicked!") # Debug print 1 - Function entry

  if file_list is None:
    print("No file uploaded!") # Debug print 2 - No file condition
    return gr.Warning("No document uploaded!")

  print("File uploaded:", file_list.name) # Debug print 3 - File name
  file_path = file_list.name # Access the uploaded file path
  try:
    print("Loading document...") # Debug print 4 - Before load_document
    document = load_document(file_path)
    print("Document loaded successfully.") # Debug print 5 - After load_document

    print("Chunking document...") # Debug print 6 - Before chunk_document
    chunks = chunk_document(document)
    print("Document chunked.") # Debug print 7 - After chunk_document

    print("Creating vector store...") # Debug print 8 - Before create_vector_store
    vector_store_instance = create_vector_store(chunks, embeddings)
    print("Vector store created.") # Debug print 9 - After create_vector_store

    print("Initializing chatbot...") # Debug print 10 - Before initialize_chatbot
    qa_chain_instance = initialize_chatbot(vector_store_instance)
    print("Chatbot initialized.") # Debug print 11 - After initialize_chatbot

    print("Document processed and chatbot initialized successfully!") # Debug print 12 - Success
    return gr.Info("Document processed and chatbot initialized!")
  except Exception as e:
    print(f"Error processing document: {e}") # Debug print 13 - Error catch
    return gr.Error(f"Error processing document: {e}")
  finally:
    print("Process document function completed.") # Debug print 14 - Function exit

def chatbot_response(query, chat_history):
  global qa_chain_instance
  if qa_chain_instance is None:
    return "Please upload and process a document first."
  response = get_chatbot_response(qa_chain_instance, query)
  return response

def clear_chat():
  global qa_chain_instance
  if qa_chain_instance and qa_chain_instance.memory:
    qa_chain_instance.memory.clear()
    return None # For Gradio ChatInterface clear, can return None to clear output


<ipython-input-6-6b42e2a2b18a>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key) # Initialize OpenAI embeddings


### UI Functionality (Gradio)

In [ ]:
def build_gradio_ui(process_document_fn, chatbot_response_fn, clear_chat_fn):
    """Build the Gradio UI for document upload, processing, and chatbot interaction."""
    with gr.Blocks() as demo:
        gr.Markdown("# RAG Chatbot Demo")
        with gr.Row():
            with gr.Column(scale=3):
                gr.Markdown("## Document Upload")
                file_output = gr.File(file_types=['.pdf'], label="Upload Document") # Example for PDF
                process_button = gr.Button("Process Document")
                process_button.click(
                    process_document_fn,
                    inputs=[file_output],
                    outputs=[] # Process function will update state, no direct output to UI yet
                )
                gr.Markdown("---")
            with gr.Column(scale=7):
                chatbot_interface = gr.ChatInterface(
                    chatbot_response_fn,
                    chatbot=None, # No custom chatbot function for now, using direct response
                    additional_inputs=[], # No additional input components
                    #clear_btn="Clear Chat",
                )
                clear = gr.ClearButton([chatbot_interface])
                #chatbot_interface.clear_btn.click(clear_chat_fn, inputs=[], outputs=[chatbot_interface]) # Clear chat button

    return demo

### Launch the UI

In [ ]:
ui = build_gradio_ui(process_document, chatbot_response, clear_chat)
ui.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:290: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3fd11ca1a3a9452833.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Process Document button clicked!
File uploaded: /tmp/gradio/20f4fb8f773ebc5826ca09f0a02dc8acd679427d7f9306882361d7436401dda6/Principal-Sample-Life-Insurance-Policy.pdf
Loading document...
Document loaded successfully.
Chunking document...
Document chunked.
Creating vector store...
Vector store created.
Initializing chatbot...
Chatbot initialized.
Document processed and chatbot initialized successfully!
Process document function completed.


<ipython-input-5-502d66ff3204>:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
